# Livrable 1 - classification d'images

## Dépendances

In [ ]:
#Import des dépendances
import os
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf


from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping

# Analyse des données

## Import des données

In [ ]:
# Chemin vers le dataset
dataset_path = "./Dataset/Dataset1" # Chemin vers le dataset à modifier

# Vérification de l'existence du dossier
if not os.path.exists(dataset_path):
    raise FileNotFoundError(f"Le dossier {dataset_path} n'existe pas.")

# Récupération des classes et du nombre d'images par classe
classes = []
image_counts = []

# Parcours des fichiers dans le dataset
for class_name in os.listdir(dataset_path):  # Parcours des fichiers dans le dataset
    class_path = os.path.join(dataset_path, class_name)  # Chemin vers le fichier
    classes.append(class_name.replace('Dataset Livrable 1 - ',''))  # Ajout du nom de la classe sans l'extension
    image_counts.append(len(os.listdir(class_path)))  # Comptage des fichiers dans le dossier

## Exploration des données

In [ ]:
# Affichage des classes et du nombre d'images par classe
print("Classes : ", classes)
print("Nombre d'images par classe : ", image_counts)

# Affichage de l'histogramme
plt.figure(figsize=(10, 6))
plt.bar(classes, image_counts, color='skyblue')
plt.xlabel('Classes')
plt.ylabel('Nombre d\'images')
plt.title('Répartition du nombre de données (images) par classe')
plt.xticks(rotation=45)
plt.show()

In [ ]:
from collections import defaultdict
import imghdr
from PIL import Image


# Dictionnaire pour stocker le nombre d'images corrompues par classe
corrupted_count_by_class = defaultdict(int)

print("Début de la vérification des images ...")

# Parcours des fichiers ZIP
for dir_name in os.listdir(dataset_path):
    dir_path = os.path.join(dataset_path, dir_name)
    for file_name in os.listdir(dir_path):
                # Vérification si le fichier est bien une image
                if file_name.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                    try:
                        # Ouverture de l'image pour vérifier si elle est corrompue
                        with open(os.path.join(dir_path, file_name), 'rb') as file:
                            img_bytes = file.read()  # Lire les bytes de l'image
                            img = tf.image.decode_image(img_bytes)  # Essayer de décoder l'image
                    except Exception as e:
                        print(f"Image {file_name} dans {dir_name} est corrompue. Exception: {e}")
                        corrupted_count_by_class[dir_name] += 1
                else:
                    print(f"Le fichier {file_name} dans {dir_name} n'est pas une image.")
                    corrupted_count_by_class[dir_name] += 1

print("Vérification des images dans les fichiers zip terminée.")

# Affichage du nombre d'images corrompues par fichier ZIP
for dir_name, count in corrupted_count_by_class.items():
    print(f"Dossier {dir_name} : {count} images corrompues")

# Nombre total d'images corrompues
total_corrupted = sum(corrupted_count_by_class.values())
print(f"Nombre total d'images corrompues : {total_corrupted}")


In [ ]:
#Afficher les n premières images de x
def display_image(X, n):
    plt.figure(figsize=(20, 2))
    for i in range(n):
        ax = plt.subplot(1, n, i + 1)
        plt.imshow(X[i].reshape(28, 28), cmap='gray')
        #plt.axis('off')
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    plt.show()

## Nettoyage des données

## Ajout de donnée avec GAN

In [ ]:
# Définition des dimensions de l'image et du bruit
img_height, img_width = 64, 64
channels = 1  # Pour des images en niveaux de gris (sketch)
noise_dim = 100  # Dimension du vecteur de bruit

# Générateur
def build_generator():
    model = Sequential([
        layers.Dense(8 * 8 * 256, use_bias=False, input_shape=(noise_dim,)),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Reshape((8, 8, 256)),
        layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Conv2DTranspose(channels, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh')
    ])
    return model

# Discriminateur
def build_discriminator():
    model = Sequential([
        layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same', input_shape=(img_height, img_width, channels)),
        layers.LeakyReLU(),
        layers.Dropout(0.3),
        layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'),
        layers.LeakyReLU(),
        layers.Dropout(0.3),
        layers.Flatten(),
        layers.Dense(1)
    ])
    return model

# Compilation des modèles
generator = build_generator()
discriminator = build_discriminator()

cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

# Entraînement du GAN
@tf.function
def train_step(images):
    noise = tf.random.normal([batch_size, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

# Boucle d'entraînement
def train(dataset, epochs):
    for epoch in range(epochs):
        for image_batch in dataset:
            train_step(image_batch)

        # Génération d'images après chaque epoch
        noise = tf.random.normal([16, noise_dim])
        generated_images = generator(noise, training=False)

        # Affichage des images générées
        plt.figure(figsize=(4, 4))
        for i in range(generated_images.shape[0]):
            plt.subplot(4, 4, i + 1)
            plt.imshow(generated_images[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
            plt.axis('off')
        plt.show()

# Préparation des données pour la classe "sketch"
sketch_path = os.path.join(dataset_path, "sketch")
sketch_images = []
for file_name in os.listdir(sketch_path):
    img = Image.open(os.path.join(sketch_path, file_name)).convert('L').resize((img_height, img_width))
    sketch_images.append(np.array(img) / 127.5 - 1)  # Normalisation entre -1 et 1
sketch_images = np.expand_dims(sketch_images, axis=-1)
sketch_dataset = tf.data.Dataset.from_tensor_slices(sketch_images).shuffle(1000).batch(32)

# Entraînement du GAN
train(sketch_dataset, epochs=50)

## Affichage d'échantillon de data

# Pipeline Classification

# Pipeline Denoising

# Pipeline Captioning